# Creation of the Dataset

### Importations 

In [63]:
import numpy as np
import matplotlib.pyplot as plt
import IPython
import pyroomacoustics as pra
import pickle as pkl
import pandas as pd
from scipy.io import wavfile
from scipy.signal import fftconvolve
from scipy import signal
from matplotlib import gridspec

In [2]:
def compute_absorption(room_eadge,room_height,RT60):
    V = (room_eadge**2)*room_height
    S = 2*(room_eadge**2) + 4*(room_height**2)
    absr = 0.1611*(V/(S*RT60))
    return absr

### Setting the different parameters for the rooms

In [4]:
#Here You should put the path to where the file wav is on the machine you are working on.
folder_path = "/home/janjar/Semester_Project/Dataset/GDWN/WGN.wav"
file_to_open = folder_path 

ang = (np.array([61.]) / 180.) * np.pi
distance = 1.0
a = np.array([5.,5.,3.])
b = np.array([6.,6.,3.])
dimensions = np.stack((a,b))
distances = np.array([1.,2.])
RT60 = np.array([0.2,0.3])

absorption_1 = compute_absorption(a[0],a[2],RT60[0])
absorption_2 = compute_absorption(b[0],b[2],RT60[1])
absorptions = np.array([absorption_1,absorption_2])
print(absorptions.shape)

snr = 5.
fs,signal_wav = wavfile.read(file_to_open)
snrs = np.array([5,10,15,20])

(2,)


In [64]:
def generate_room_from_conditions(ang,distance,absorptions,snr,dimensions,typeofroom):
   
    room_dim = dimensions[typeofroom]
    room_eadge = room_dim[0]

    sf = (0.03*np.sqrt(2)/4)
    corners = np.array([[0,0],[0,room_eadge],[room_eadge,room_eadge],[room_eadge,0]]).T
    
    absr = absorptions[typeofroom]
    room = pra.Room.from_corners(corners, fs=fs , max_order=8 ,absorption=absorptions[typeofroom])
    room.extrude(3.)

    # Add sources of 1 second duration
    #room_dim = np.array([5,5,3])
    rng = np.random.RandomState(23)
    duration_samples = int(fs)


    source_location = room_dim / 2 + (distance * np.array([np.cos(ang), np.sin(ang),0.0]))
    source_signal = rng.randn(duration_samples)
    room.add_source(source_location, signal = signal_wav)
    print('Here is the source at this step:',source_location) 

    #We initiate the point of the Tethra then we scale them
    #and translate the origin of the Tethra to the center of the room.
    R = np.array([(sf*np.array([1,1,-1,-1]))+2.5,(sf*np.array([1,-1,1,-1]))+2.5,(sf*np.array([1,-1,-1,1]))+1.5])# [[x], [y], [z]]
    room.add_microphone_array(pra.MicrophoneArray(R,room.fs))
    room.image_source_model(use_libroom=True)

    #Visualization
    #fig, ax = room.plot(img_order=3)
    #fig.set_size_inches(18.5,10.5)

    room.simulate(snr = snr)
    #print('Here is the audio signal:',room.mic_array.signals[0][:5])
    return room.mic_array.signals,source_location,absr

### Testing that generate_room works properly

In [44]:
ang = (np.array([61.]) / 180.) * np.pi
audio_signals,source_loc,t = generate_room_from_conditions(ang,distances[0],absorptions,snr,dimensions,0)
print(audio_signals[0,:].shape)

Here is the source at this step: [array([2.98480962]) array([3.37461971]) 1.5]
Here is the audio signal: [-0.78211052  0.31446528  0.23889686 -0.18525975  0.5302637 ]
(49702,)


In [65]:
def generate_phasematrix_from_signals(signals,j):
    
    #Here also the path should be the folder you need the audio files to be stored on.
        path = "/home/janjar/Dataset/Trainingset/"   
        name_signals = 'audio_signals/audio_signals-{}'.format(j)
        fileName_audio = path + name_signals
        fileObject = open(fileName_audio, 'wb')
        pkl.dump(signals, fileObject)
        fileObject.close()                
    
        phase_matrix = np.empty((4, 129, 390))
        for i in range(4):
            f, t, stft_mic0 = signal.stft(audio_signals[i,:].astype(np.float32), fs)
            spectrum = stft_mic0
            magnitude = np.abs(spectrum)
            phase = np.angle(spectrum)
            #print('Here is the phase:',phase)
            x = phase.shape[0]
            y = phase.shape[1]
            #print("HERE IS THE SHAPE",phase.shape)
            #phase_matrix = np.empty((4, x, y))
            phase_matrix[i] = phase
            
        path = "/home/janjar/Dataset/Trainingset/"   
        name_matrix = 'phase_matrix/Phase_matrix-{}'.format(j)
        fileName_matrix = path + name_matrix
        fileObject = open(fileName_matrix, 'wb')
        pkl.dump(phase_matrix, fileObject)
        fileObject.close()                    
        return fileName_audio,fileName_matrix

### Testing that generate_phasematrix works properly

In [66]:
path_to_matrix = generate_phasematrix_from_signals(audio_signals,0)
print(path_to_matrix)

('/home/janjar/Dataset/Trainingset/audio_signals/audio_signals-0', '/home/janjar/Dataset/Trainingset/phase_matrix/Phase_matrix-0')


In [67]:
def assign_label_to_anlges(angles):
	bins = np.linspace(0, 180, 37)
	label = np.digitize(angles, bins)
	return label

In [68]:
def genere_dataset():
        index = 0
        df = pd.DataFrame(columns = ['Room','Array_position','Distance','Absorption','SNR','Audio_file','Phase_Matrix','Label'])
        snr = np.array([0,5,10,15,20]) #Comment this part to turn the snr into random assignment for each room
	#We iterate over the type of rooms (R1,R2).
        for i in range (2):          
	#We iterate over the distances from the microphones (1m,2m).	
            for j in range(2):
    #We iterate over the 5 different SNRS.       
                for l in range(5):    
	#We iterate on the 7 random positions of the source.		
                    for k in range(7):
                        print('Room:',index)
                        angles = np.random.uniform(low=0, high=180, size=(7,))
                        #print('Here are the angles for this Room:',angles) #Make the angles able to go to floats.
                        labels = assign_label_to_anlges(angles)
                        #print('Here are the labels for this Room:',labels) #Check if the labels match the angles.
                        azimuth = angles / 180. * np.pi 
                        #snr = snrs[np.random.randint(4,size = 1)] # Uncomment this part to turn the snr into random assignment for each room
                        signals,array_position,absr = generate_room_from_conditions(angles[k],distances[j],absorptions,snr[l],dimensions,i)
                        fileName_audio,fileName_matrix = generate_phasematrix_from_signals(signals,index)
                        df = df.append({'Room':i,'Array_position':array_position,'Distance':distances[j],'Absorption':absr,'SNR':snr[l],'Audio_file':fileName_audio,'Phase_Matrix':fileName_matrix,'Label':labels[k]} , ignore_index=True)
                        index = index+1
        return df               



In [69]:
Dataset = genere_dataset()

Room: 0
Here is the source at this step: [1.64729108 3.02238635 1.5       ]
Room: 1
Here is the source at this step: [3.44627959 2.17665045 1.5       ]
Room: 2
Here is the source at this step: [1.69882607 3.09843156 1.5       ]
Room: 3
Here is the source at this step: [3.35208369 3.02340557 1.5       ]
Room: 4
Here is the source at this step: [3.49575878 2.59200248 1.5       ]
Room: 5
Here is the source at this step: [3.3654378  1.99898361 1.5       ]
Room: 6
Here is the source at this step: [1.66529112 1.94930855 1.5       ]
Room: 7
Here is the source at this step: [3.39739831 2.05877866 1.5       ]
Room: 8
Here is the source at this step: [2.24340451 1.53348111 1.5       ]
Room: 9
Here is the source at this step: [2.87852832 3.42558971 1.5       ]
Room: 10
Here is the source at this step: [2.74108434 1.52949583 1.5       ]
Room: 11
Here is the source at this step: [2.57894047 1.50312067 1.5       ]
Room: 12
Here is the source at this step: [3.32865835 1.94024529 1.5       ]
Room: 13


Room: 108
Here is the source at this step: [4.84728765 3.76650397 1.5       ]
Room: 109
Here is the source at this step: [1.40542431 4.20719856 1.5       ]
Room: 110
Here is the source at this step: [4.99901876 2.93735831 1.5       ]
Room: 111
Here is the source at this step: [1.08199263 3.56678719 1.5       ]
Room: 112
Here is the source at this step: [4.9033669  2.38583844 1.5       ]
Room: 113
Here is the source at this step: [3.73976469 4.85815721 1.5       ]
Room: 114
Here is the source at this step: [4.18833921 4.60867956 1.5       ]
Room: 115
Here is the source at this step: [3.44635076 1.05044338 1.5       ]
Room: 116
Here is the source at this step: [1.10788585 2.35199997 1.5       ]
Room: 117
Here is the source at this step: [4.72318699 1.98480218 1.5       ]
Room: 118
Here is the source at this step: [2.58851047 4.95721138 1.5       ]
Room: 119
Here is the source at this step: [4.73095287 4.00189929 1.5       ]
Room: 120
Here is the source at this step: [1.1396329  3.7341895

In [70]:
Dataset

,Room,Array_position,Distance,Absorption,SNR,Audio_file,Phase_Matrix,Label
0,0,"[1.6472910781788257, 3.0223863461525102, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,31
1,0,"[3.446279591532051, 2.17665044510633, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,16
2,0,"[1.6988260699608255, 3.098431561521937, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,34
3,0,"[3.352083687061692, 3.0234055695570623, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,13
4,0,"[3.4957587779161745, 2.592002479330104, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,34
5,0,"[3.3654377965721802, 1.998983612778694, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,4
6,0,"[1.6652911157336703, 1.9493085450755032, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,23
7,0,"[3.3973983118920215, 2.0587786611989958, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,7
8,0,"[2.243404513484258, 1.5334811143595022, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,13
9,0,"[2.8785283178590926, 3.425589710713535, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,2


In [72]:
Dataset.to_pickle("/home/janjar/Dataset/Trainingset/Training_dataframe.pkl")